
# Tutorial

#Creating your first ChatBot using Bert

In [ ]:
#install relevant libraries

!pip install transformers==4.10
!pip install datasets

     |████████████████████████████████| 2.8 MB 13.7 MB/s 
     |████████████████████████████████| 895 kB 81.7 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 55.2 MB/s 
     |████████████████████████████████| 596 kB 72.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 311 kB 13.2 MB/s 
     |████████████████████████████████| 133 kB 71.4 MB/s 
     |████████████████████████████████| 1.1 MB 70.8 MB/s 
     |████████████████████████████████| 243 kB 86.9 MB/s 
     |████████████████████████████████| 160 kB 86.2 MB/s 
     |████████████████████████████████| 144 kB 72.4 MB/s 
     |████████████████████████████████| 271 kB 65.5 MB/s 


In [ ]:
#import packages

import pandas as pd
import numpy as np
import random
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
import transformers
import logging
import os
import random
import sys
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import is_main_process
from datasets import load_dataset, load_metric,Dataset
from sklearn.metrics import accuracy_score

import json
import string
import random 
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
%cd /content/drive/MyDrive/Tutorial/
%ls

/content/drive/MyDrive/Tutorial
checkpoint/  Intent.json  log/  model/  student_notebook.ipynb  Training.ipynb


In [ ]:
#import your data "json" file
import json 

temp_data = pd.DataFrame()
file_object = open('Intent.json')
input_json = json.loads(file_object.read())

In [ ]:
#convert your json file into dataframe for easy handling

intent_label = []
sentences = []
responses = {}

for d in input_json['intents']:
    responses[d['intent']] = d['responses']
    texts = d['text']
    sentences.extend(texts)
    intent_label.extend([d['intent']]*len(texts))

temp_data['sentences'] = sentences
temp_data['intent_label'] = intent_label

In [ ]:
#assign an integer to your labels for the understanding of the model

int_label = {}
reverse = {}
for i,intent in enumerate(list(set(intent_label))):
    int_label[intent] = i
    reverse[i] = intent 

label = []
for index,row in temp_data.iterrows():
    label.append(int_label[row['intent_label']])

temp_data['label'] = label

In [ ]:
#import the tokenizer to tokenize sentences into numbers for the understanding of the model

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#create a preprocess function that uses the tokenizer to tokenize sentences.

def preprocess_function(examples):
        # Tokenize the texts
        args = examples["sentences"]
        result = tokenizer(args, padding=True, truncation=True)
        result["input_ids"] = np.array(result["input_ids"], dtype="int64")
        result["attention_mask"] = np.array(result["attention_mask"], dtype="int64")
        result["label"] = [i for i in examples["label"]]
        #print(result.keys())
        return result

#prepare the dataset

training_set = Dataset.from_pandas(temp_data)

training_loader = training_set.map(preprocess_function, batched=True,load_from_cache_file=False)
columns_to_return = ['input_ids', 'label', 'attention_mask']
training_loader.set_format(type='torch', columns=columns_to_return)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
#load the model

config = AutoConfig.from_pretrained('distilbert-base-uncased',num_labels = len(set(label)))
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased',config=config)

#no finetuning
for param in model.base_model.parameters():
          param.requires_grad = False

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [ ]:
#initialise the training setup

training_args = TrainingArguments(
            output_dir= 'checkpoint/',          # output directory
            num_train_epochs=50,              # total number of training epochs
            per_device_train_batch_size=1,  # batch size per device during training   
            logging_dir='log/',            # directory for storing logs
            logging_steps=10, 
            save_steps = 500,
            warmup_steps=500,                # number of warmup steps for learning rate scheduler
            weight_decay=0.01,
            )
trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=training_loader,
            data_collator= None)
trainer.train()

In [ ]:
#save your model
trainer.save_model('/content/drive/MyDrive/Tutorial/model/')

In [ ]:
#load the model

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
config = AutoConfig.from_pretrained('/content/drive/MyDrive/Tutorial/model/',num_labels = len(set(label)))
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/Tutorial/model/',config=config)

In [ ]:
#build the pipeline
from transformers import pipeline
pipe = pipeline("text-classification",model=model,tokenizer=tokenizer)

In [ ]:
pipe("Hi")

[{'label': 'LABEL_12', 'score': 0.3560907542705536}]

In [ ]:
#get response function

import random
def get_response(sentence): 
  return random.sample(responses[reverse[int(pipe([sentence])[0]['label'].split('_')[-1])]],1)[0].replace('<HUMAN>','Elon')

In [ ]:
temp_data['sentences'].tolist()

['Hi',
 'Hi there',
 'Hola',
 'Hello',
 'Hello there',
 'Hya',
 'Hya there',
 'My user is Adam',
 'This is Adam',
 'I am Adam',
 'It is Adam',
 'My user is Bella',
 'This is Bella',
 'I am Bella',
 'It is Bella',
 'How are you?',
 'Hi how are you?',
 'Hello how are you?',
 'Hola how are you?',
 'How are you doing?',
 'Hope you are doing well?',
 'Hello hope you are doing well?',
 'Good thanks! My user is Adam',
 'Good thanks! This is Adam',
 'Good thanks! I am Adam',
 'Good thanks! It is Adam',
 'Great thanks! My user is Bella',
 'Great thanks! This is Bella',
 'Great thanks! I am Bella',
 'Great thanks! It is Bella',
 'What is my name?',
 'What do you call me?',
 'Who do you think I am?',
 'What do you think I am?',
 'Who are you talking to?',
 'What name do you call me by?',
 'Tell me my name',
 'What is your name?',
 'What could I call you?',
 'What can I call you?',
 'What do your friends call you?',
 'Who are you?',
 'Tell me your name?',
 'What is your real name?',
 'What is your

In [ ]:
reverse = {0: 'Gossip',
 1: 'Thanks',
 2: 'UnderstandQuery',
 3: 'Clever',
 4: 'Shutup',
 5: 'RealNameQuery',
 6: 'SelfAware',
 7: 'CourtesyGreetingResponse',
 8: 'NotTalking2U',
 9: 'Swearing',
 10: 'GreetingResponse',
 11: 'CourtesyGreeting',
 12: 'Greeting',
 13: 'Jokes',
 14: 'PodBayDoor',
 15: 'CurrentHumanQuery',
 16: 'PodBayDoorResponse',
 17: 'NameQuery',
 18: 'GoodBye',
 19: 'CourtesyGoodBye',
 20: 'TimeQuery',
 21: 'WhoAmI'}

In [ ]:
# running the chatbot
while True:
    message = input("")
    result = get_response(message)
    print(result)

Hi
Hola human, please tell me your GeniSys user
Can you prove you are self-aware
That depends, can you prove that you are?


KeyboardInterrupt: ignored